In [1]:
import pandas as pd
import plotly.express as px

print("✅ Entorno conectado correctamente.")

✅ Entorno conectado correctamente.


In [2]:
import pandas as pd
import plotly.express as px

# 1. CARGA DE DATOS (Copiamos la lógica base)
url_bici = "https://ckan-data.montevideo.gub.uy/dataset/15a3de29-2353-4d5f-9bae-e1617fa9e974/resource/6ada7fb7-cd03-4752-8b73-ef083df739aa/download/conteo_ciclovia_2025.csv"
df = pd.read_csv(url_bici, encoding='utf-8', on_bad_lines='skip')
df.columns = df.columns.str.strip().str.lower()

# Procesamiento
df['fecha_dt'] = pd.to_datetime(df['fecha'], dayfirst=True, errors='coerce')
df['hora_num'] = pd.to_numeric(df['hora'].astype(str).str.split(':').str[0], errors='coerce')
df['total_bicis'] = df['hacia_ciudad_vieja'] + df['hacia_tres_cruces']

# 2. PREPARACIÓN PARA EL MAPA DE CALOR
# Extraemos el nombre del día y lo ordenamos para que no salga alfabético
df['dia_semana'] = df['fecha_dt'].dt.day_name()
orden_dias = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
mapa_espanol = {'Monday': 'Lunes', 'Tuesday': 'Martes', 'Wednesday': 'Miércoles', 'Thursday': 'Jueves', 
                'Friday': 'Viernes', 'Saturday': 'Sábado', 'Sunday': 'Domingo'}

# Agrupamos: Promedio de bicis por (Día, Hora)
# Esto nos dice "El Lunes a las 18hs pasan X bicis en promedio"
heatmap_data = df.groupby(['dia_semana', 'hora_num'])['total_bicis'].mean().reset_index()

# Aplicamos traducción y orden
heatmap_data['dia_label'] = heatmap_data['dia_semana'].map(mapa_espanol)

# 3. GRAFICAR (DENSITY HEATMAP)
fig = px.density_heatmap(
    heatmap_data, 
    x='hora_num', 
    y='dia_semana', # Usamos el inglés para ordenar fácil, luego cambiamos la etiqueta
    z='total_bicis', 
    nbinsx=24, 
    title='🔥 Mapa de Calor: Intensidad de Uso Semanal',
    category_orders={'dia_semana': orden_dias}, # Forza el orden Lunes-Domingo
    labels={'dia_semana': 'Día', 'hora_num': 'Hora', 'total_bicis': 'Intensidad'},
    color_continuous_scale='Viridis' # Escala de colores profesional
)

# Truco para que en el eje Y salgan los nombres en español
fig.update_yaxes(tickmode='array', tickvals=orden_dias, ticktext=list(mapa_espanol.values()))

fig.show()

In [ ]:
# ANÁLISIS AUTOMÁTICO DE SIGNOS VITALES
stats = df['total_bicis'].describe()

media = stats['mean']
maximo = stats['max']
std = stats['std']
mediana = stats['50%']

print("📝 CONCLUSIONES DEL INGENIERO (GENERADAS POR PYTHON):")
print("-" * 60)

# 1. Análisis de Capacidad
if maximo > (media * 3):
    print(f"⚠️ ALERTA DE ESTRÉS: El sistema soporta picos extremos.")
    print(f"   El récord ({int(maximo)} bicis) es más del triple que el promedio ({int(media)}).")
    print("   -> Sugerencia: Revisar ancho de vía para horas pico.")
else:
    print(f"✅ FLUJO ESTABLE: No hay picos dramáticos respecto al promedio.")

# 2. Análisis de Realidad (Mediana vs Promedio)
if media > mediana * 1.5:
    print(f"\n📊 DATO CURIOSO: El promedio está inflado por pocos días 'locos'.")
    print(f"   La realidad (mediana) es {int(mediana)} bicis, pero el promedio dice {int(media)}.")
else:
    print(f"\n✅ DATO: El flujo es bastante consistente día a día.")

print("-" * 60)

92.42190640252656
📝 CONCLUSIONES DEL INGENIERO (GENERADAS POR PYTHON):
------------------------------------------------------------
⚠️ ALERTA DE ESTRÉS: El sistema soporta picos extremos.
   El récord (500 bicis) es más del triple que el promedio (92).
   -> Sugerencia: Revisar ancho de vía para horas pico.

✅ DATO: El flujo es bastante consistente día a día.
------------------------------------------------------------
